## Goal: get SNPs both CLOSE to SJs (exonic, +- 10 bp) and within SSs themselves

* 1. Use these to compute what % of novel SJs only in hap1/hap2 have a +- 10bp SNP OR SNP in the SS (should be very close to another I'm computing) AND
* 2. Subset on just known SJs to compute a known SJ-proximal SNP density score that I can use to correlate against the % increase in ICs / SJs detected to predict how much novel splicing we're missing in populations / samples that we actually didn't profile here

In [40]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot

p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *

In [41]:
config = load_config()
od = '../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [42]:
df = load_meta()
df = df.loc[~df['sample'].str.contains('_')]
df['lab_sample'] = df['lab_number_sample'].astype(str)+'_'+\
                      df['lab_sampleid'].astype(str)+'_'+\
                      df['cell_line_id'].astype(str)
df.columns
df = df[['cell_line_id', 'sample', 'hapmap_DNA_ID',
          'map_reads_assemblymap','lab_sample', 'population']].drop_duplicates()

temp_df = pd.read_csv('../snakemake/transcript_discovery_personal/cell_line_ids.txt', header=None, names=['cell_line_id'])

# make a 1000g cell line id col
df['cell_line_id_1000g'] = df.cell_line_id

inds = df.loc[~df.cell_line_id_1000g.isin(temp_df.cell_line_id.tolist())].index
df.loc[inds, 'cell_line_id_1000g'] = df.loc[inds, 'hapmap_DNA_ID']
len(df.index)

# limit to just those in 1000g
df = df.loc[df.cell_line_id_1000g.isin(temp_df.cell_line_id.tolist())]
assert len(df.index) == 30

# TODO bad sample that hasn't finished on espresso
# bad_samples = ['NA19328']
# df = df.loc[~df.cell_line_id_1000g.isin(bad_samples)]

hap = ['hap1', 'hap2']

meta_df = df.copy(deep=True)

## We want to know what % of 1.5 NOVEL sjs that we 1. discover only in personal haplotypes that have snps +- 10 of splice junctions OR in splice sites

In [45]:
file = proc_cfg(config['lr']['td_personal']['sqanti']['sj_summary'],od)
df = pd.read_csv(file)
df.drop('Unnamed: 0', axis=1, inplace=True)
df['detected'] = True

In [46]:
# 1.5 only novel sjs
temp = df.loc[df.junction_novelty=='novel']

In [ ]:
# get only the detection from mapping
temp = temp[['sj_id', 'cell_line_id', 'map_genome', 'detected']]
temp = temp.drop_duplicates()
temp = temp.pivot(index=['sj_id', 'cell_line_id'], 
                columns=['map_genome'],
                values=['detected'])

# flatten
temp.columns = temp.columns.get_level_values(1)
temp.columns.name = None

# reset index to make it a flat DataFrame
temp = temp.reset_index()

# fill missing values with False
temp = temp.fillna(False)

In [ ]:
# 1. get those that are detected uniquely in hap1/2 
# (ie those not in hg38)
temp = temp.loc[(temp.hg38==False)&
                ((temp.hap1+temp.hap2)>0)]

In [ ]:
# 3. process snp content in each sj
sj_snp_df = pd.DataFrame()
for c in meta_df['cell_line_id_1000g'].tolist():
    file = proc_cfg(expand(config['lr']['td_personal']['exon_vars']['sj_12nt_vcf_intersect'],
                cell_line_id=c)[0],
                od)
    df = pr.read_bed(file).df
    df = df.drop_duplicates()
    
    # just keep uniq. sj_ids
    df.rename({'ThickStart': 'sj_id'}, axis=1, inplace=True)
    df = df[['sj_id']]
    
    df['cell_line_id'] = c
    df['exon_12nt_has_var'] = True
    sj_snp_df = pd.concat([sj_snp_df, df], axis=0)
sj_snp_df.head()

In [ ]:
# 3. process snp content in each sj
sj_snp_df = pd.DataFrame()
for c in meta_df['cell_line_id_1000g'].tolist():
    file = proc_cfg(expand(config['lr']['td_personal']['exon_vars']['sj_10nt_vcf_intersect'],
                cell_line_id=c)[0],
                od)
    df = pr.read_bed(file).df
    df = df.drop_duplicates()
    
    # just keep uniq. sj_ids
    df.rename({'ThickStart': 'sj_id'}, axis=1, inplace=True)
    df = df[['sj_id']]
    
    df['cell_line_id'] = c
    df['exon_12nt_has_var'] = True
    sj_snp_df = pd.concat([sj_snp_df, df], axis=0)
sj_snp_df.head()

In [ ]:
# compute the # and % of hap1/hap2-spec sjs (per sample) that hav variants in +-10bp + SSs
temp = temp[['sj_id', 'cell_line_id', 'exon_12nt_has_var']]
temp = temp.groupby(['cell_line_id', 'exon_12nt_has_var']).nunique().reset_index().rename({'sj_id':'n_sj'}, axis=1)

In [ ]:
# add total & compute %
temp['total'] = temp.groupby('cell_line_id')['n_sj'].transform('sum')
temp['perc'] = (temp.n_sj/temp.total)*100

In [ ]:
init_plot_settings(aspect='square')
temp = temp.loc[temp.has_explanation==True]
ax  = sns.boxplot(temp, 
                  y='perc', showfliers=False)
sns.stripplot(temp,
              y='perc',
              size=10)
ax.set(ylabel='% of novel, hg38 undet. SJs\nwhich have a variant either\nin SS or exonic +-10bp')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

## Dev for getting the +- 10 + SS bed file

In [4]:
file = proc_cfg(config['lr']['td_personal']['sqanti']['sj_summary'],od)
df = pd.read_csv(file)

24934518
8311506


In [ ]:
temp_back = temp.copy(deep=True)

In [38]:

temp = temp_back.copy(deep=True)
# check w/ just 1 genome + cell line at first so I can 
# dl GTF and check it
temp = temp.loc[(temp.cell_line_id=='HG03732')&\
                (temp.hap1==True)]


In [39]:
df.drop('Unnamed: 0', axis=1, inplace=True)

# remove sqanti genome and drop dupes
# the sqanti genome / sqanti metrics SHOULD be irrelevant here
df = df.drop(['sqanti_genome', 'canonical', 'splice_motif'], axis=1)
print(len(df.index))
df = df.drop_duplicates()
print(len(df.index))

# then make sure that there are no dupe. sj+sj nov+sample+map genome
temp = df.loc[df[['sj_id', 'junction_novelty',
                  'cell_line_id', 'map_genome', 'start_site_novelty',
                  'end_site_category']].duplicated(keep=False)]
assert len(temp.index) == 0
del temp

df.rename({'end_site_category': 'end_site_novelty'}, axis=1, inplace=True)

# transform to be t/f for each ic per genome
temp = pd.crosstab(index=[df.sj_id, df.junction_novelty,
                                 df.start_site_novelty,
                                 df.end_site_novelty, df.cell_line_id],
                          columns=df.map_genome,
                          values=df.map_genome,
                          aggfunc=lambda x: True).fillna(False).reset_index()


temp[['Chromosome', 'Strand', 'Start', 'End']] = temp.sj_id.str.split('_', expand=True)
temp = temp[['Chromosome', 'Strand', 'Start', 'End',
             'cell_line_id', 'sj_id']].drop_duplicates()

temp.Start = temp.Start.astype(int)
temp.End = temp.End.astype(int)
assert len(temp.loc[temp.Start>temp.End])==0

# melt to 5' and 3'
temp = temp.melt(id_vars=['Chromosome', 'Strand', 'cell_line_id', 'sj_id'],
                 value_vars=['Start', 'End'])
temp['sj_loc'] = ''
temp.loc[temp.map_genome=='Start', 'sj_loc'] = 'start'
temp.loc[temp.map_genome=='End', 'sj_loc'] = 'end'

temp.rename({'value':'Start'}, axis=1, inplace=True)
# need to verify that this is working using like one motif or something make
# sure I don't have off-by-one errors
# verified
temp['Start'] = temp.Start-2
temp['End'] = temp.Start+1

# verified https://trello.com/c/fMhwX3s6
temp.loc[temp.sj_loc=='start', 'Start'] = temp.loc[temp.sj_loc=='start', 'Start']-9
temp.loc[temp.sj_loc=='start', 'End'] = temp.loc[temp.sj_loc=='start', 'End']+2


temp.loc[temp.sj_loc=='end', 'End'] = temp.loc[temp.sj_loc=='end', 'End']+11
temp.loc[temp.sj_loc=='end', 'Start'] = temp.loc[temp.sj_loc=='end', 'Start']

temp.drop(['cell_line_id', 'map_genome'], axis=1, inplace=True)

temp['len'] = temp.End-temp.Start
assert len(temp.loc[temp.len!=12]) == 0

temp = pr.PyRanges(temp)

In [37]:
temp.to_bed('test_4.bed')
